In [1]:
# Imports
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import ZFeatureMap, ZZFeatureMap, TwoLocal, RealAmplitudes, EfficientSU2
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit.quantum_info import SparsePauliOp
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import time
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary as sel
from typing import List, Dict, Optional, Tuple
from qiskit.transpiler import PassManager, CouplingMap
from qiskit.transpiler.passes import BasicSwap, TrivialLayout, SabreSwap, SabreLayout, UnrollCustomDefinitions, BasisTranslator
from qiskit import transpile

In [ ]:
# Quantum Kernel Circuits with different entanglement strategies for ZZFeatureMap

def generate_zz_linear_kernel(num_qubits, reps=1):

    feature_map = ZZFeatureMap(feature_dimension=num_qubits, entanglement='linear', reps=reps)
    circuit = QuantumCircuit(num_qubits)
    circuit.compose(feature_map, inplace=True)
    circuit.compose(feature_map.inverse(), inplace=True)
    return circuit

def generate_zz_circular_kernel(num_qubits, reps=1):

    feature_map = ZZFeatureMap(feature_dimension=num_qubits, entanglement='circular', reps=reps)
    circuit = QuantumCircuit(num_qubits)
    circuit.compose(feature_map, inplace=True)
    circuit.compose(feature_map.inverse(), inplace=True)
    return circuit

def generate_zz_sca_kernel(num_qubits, reps=1):

    feature_map = ZZFeatureMap(feature_dimension=num_qubits, entanglement='sca', reps=reps)
    circuit = QuantumCircuit(num_qubits)
    circuit.compose(feature_map, inplace=True)
    circuit.compose(feature_map.inverse(), inplace=True)
    return circuit

def generate_zz_pairwise_kernel(num_qubits, reps=1):

    feature_map = ZZFeatureMap(feature_dimension=num_qubits, entanglement='pairwise', reps=reps)
    circuit = QuantumCircuit(num_qubits)
    circuit.compose(feature_map, inplace=True)
    circuit.compose(feature_map.inverse(), inplace=True)
    return circuit

# QNN circuits with fixed ZZFeatureMap (linear entanglement) but varying TwoLocal entanglement

def generate_qnn_linear(num_qubits, ansatz_reps=2, feature_map_reps=1):

    # Feature map with fixed linear entanglement
    feature_map = ZZFeatureMap(feature_dimension=num_qubits, entanglement='linear', reps=feature_map_reps)
    
    # Ansatz with linear entanglement
    ansatz = TwoLocal(num_qubits, entanglement='linear', reps=ansatz_reps)
    
    
    circuit = QuantumCircuit(num_qubits)
    circuit.compose(feature_map, inplace=True)
    circuit.compose(ansatz, inplace=True)
    
    return circuit

def generate_qnn_circular(num_qubits, ansatz_reps=2, feature_map_reps=1):

    # Feature map with fixed linear entanglement
    feature_map = ZZFeatureMap(feature_dimension=num_qubits, entanglement='linear', reps=feature_map_reps)
    
    # Ansatz with circular entanglement
    ansatz = TwoLocal(num_qubits, entanglement='circular', reps=ansatz_reps)
    
   
    circuit = QuantumCircuit(num_qubits)
    circuit.compose(feature_map, inplace=True)
    circuit.compose(ansatz, inplace=True)
    
    return circuit

def generate_qnn_sca(num_qubits, ansatz_reps=2, feature_map_reps=1):

    # Feature map with fixed linear entanglement
    feature_map = ZZFeatureMap(feature_dimension=num_qubits, entanglement='linear', reps=feature_map_reps)
    
    # Ansatz with sca entanglement
    ansatz = TwoLocal(num_qubits, entanglement='sca', reps=ansatz_reps)
    
    
    circuit = QuantumCircuit(num_qubits)
    circuit.compose(feature_map, inplace=True)
    circuit.compose(ansatz, inplace=True)
    
    return circuit

def generate_qnn_pairwise(num_qubits, ansatz_reps=2, feature_map_reps=1):

    # Feature map with fixed linear entanglement
    feature_map = ZZFeatureMap(feature_dimension=num_qubits, entanglement='linear', reps=feature_map_reps)
    
    # Ansatz with pairwise entanglement
    ansatz = TwoLocal(num_qubits, entanglement='pairwise', reps=ansatz_reps)
    
    
    circuit = QuantumCircuit(num_qubits)
    circuit.compose(feature_map, inplace=True)
    circuit.compose(ansatz, inplace=True)
    
    return circuit

In [ ]:
def create_tree_tensor_network(num_qubits):
    """
    Create a Tree Tensor Network (TTN) quantum circuit with logarithmic depth.
    Each layer has exactly half the number of gates as the previous layer.
    """
    # Must have a power of 2 number of qubits for a balanced tree
    if num_qubits & (num_qubits - 1) != 0:
        # If not a power of 2, round up to the next power of 2
        next_power = 2 ** np.ceil(np.log2(num_qubits))
        num_qubits = int(next_power)
    

    qc = QuantumCircuit(num_qubits)
    
    # Initialize all qubits with Hadamard gates
    for i in range(num_qubits):
        qc.h(i)
    
    qc.barrier()
    
    # Calculate number of layers (logarithmic depth)
    num_layers = int(np.log2(num_qubits))
    
    # Working qubits at each level
    active_qubits = list(range(num_qubits))
    
    # Create each layer of the tree
    for layer in range(num_layers):
        num_pairs = len(active_qubits) // 2
        new_active_qubits = []
        
        # Apply entangling gates between pairs and keep track of parent qubits
        for i in range(num_pairs):
            q1 = active_qubits[2*i]
            q2 = active_qubits[2*i + 1]
            
            # Create entanglement between pair
            qc.cx(q1, q2)
            
            qc.ry(np.pi/4, q1)
            qc.rz(np.pi/6, q2)
            
            # The first qubit of each pair becomes the "parent" for the next layer
            new_active_qubits.append(q1)
        
        qc.barrier()
        active_qubits = new_active_qubits
    
    return qc

In [ ]:
# GHZ State Circuit
def generate_ghz_state(num_qubits):

    circuit = QuantumCircuit(num_qubits)
    
    # Apply Hadamard to the first qubit
    circuit.h(0)
    
    # Apply CNOTs to entangle all qubits
    for i in range(num_qubits - 1):
        circuit.cx(i, i + 1)
    
    return circuit

In [ ]:
def create_topology(topology_type: str, num_qubits: int) -> CouplingMap:
    """
    Create a coupling map for different topology types.
    """
    edges = []
    
    if topology_type == 'linear':
        # Linear topology: each qubit connected to its neighbors
        edges = [(i, i+1) for i in range(num_qubits-1)]
        
    elif topology_type == 'ring':
        # Ring topology: linear + connection between first and last
        edges = [(i, i+1) for i in range(num_qubits-1)]
        edges.append((num_qubits-1, 0))
        
    elif topology_type == 'all':
        # All-to-all connectivity
        edges = [(i, j) for i in range(num_qubits) for j in range(i+1, num_qubits)]
        
    elif topology_type == 'grid':
        # Grid topology: arrange qubits in a square grid
        size = int(np.ceil(np.sqrt(num_qubits)))
        for i in range(num_qubits):
            row, col = i // size, i % size
            # Horizontal connections
            if col < size-1 and i+1 < num_qubits:
                edges.append((i, i+1))
            # Vertical connections
            if row < size-1 and i+size < num_qubits:
                edges.append((i, i+size))
                
    elif topology_type == 'star':
        # Star topology: central qubit (0) connected to all others
        edges = [(0, i) for i in range(1, num_qubits)]
        
    else:
        raise ValueError(f"Unknown topology type: {topology_type}")
    
    # Create bidirectional edges for bidirectional execution of 2-qubit gates
    full_edges = edges + [(j, i) for i, j in edges]
    return CouplingMap(full_edges)

def analyze_transpiled_circuit(circuit, coupling_map, basis_gates=None):
    """
    Transpiles a quantum circuit using SabreSwap and TrivialLayout with timing information.
    """
    # Calculate original circuit metrics before transpilation
    original_total_gates = sum(circuit.count_ops().values())
    original_depth = circuit.depth()
    
    # Count original two-qubit gates
    gate_counts = circuit.count_ops()
    original_two_qubit_count = sum(count for gate, count in gate_counts.items() 
                              if gate in ['cx', 'cz', 'cy', 'swap', 'iswap', 'rxx', 'ryy', 'rzz', 'ecr'])
    
    # Set default basis gates if not provided
    if basis_gates is None:
        basis_gates = ['cx', 'u3', 'swap']
    
    # Configure passes with specific parameters
    routing = SabreSwap(coupling_map=coupling_map, trials=1, heuristic='basic')
    placement = TrivialLayout(coupling_map=coupling_map)
    
    # Create passes for basis gate transformation
    unroll = UnrollCustomDefinitions(sel)
    basis_translator = BasisTranslator(sel, basis_gates)
    
    # Create pass manager with all passes
    pm = PassManager()
    pm.append([placement, routing, unroll, basis_translator])
    
    # Measure execution time
    init_time = time.monotonic()
    transpiled_circuit = pm.run(circuit)
    exec_time = time.monotonic() - init_time
    
    # Get circuit metrics after transpilation
    transpiled_total_gates = sum(transpiled_circuit.count_ops().values())
    transpiled_depth = transpiled_circuit.depth()
    swap_count = transpiled_circuit.count_ops().get('swap', 0)
    
    # Count transpiled two-qubit gate counts
    gate_counts = transpiled_circuit.count_ops()
    transpiled_two_qubit_count = sum(count for gate, count in gate_counts.items() 
                               if gate in ['cx', 'cz', 'cy', 'swap', 'iswap', 'rxx', 'ryy', 'rzz', 'ecr'])
    
    # Calculate percentage increases
    gate_increase_percentage = ((transpiled_total_gates - original_total_gates) / original_total_gates) * 100 if original_total_gates > 0 else 0
    depth_increase_percentage = ((transpiled_depth - original_depth) / original_depth) * 100 if original_depth > 0 else 0
    two_qubit_increase = ((transpiled_two_qubit_count - original_two_qubit_count) / original_two_qubit_count) * 100 if original_two_qubit_count > 0 else 0
    
    return {
        'original_depth': original_depth,
        'transpiled_depth': transpiled_depth,
        'depth_increase_percentage': depth_increase_percentage,
        'original_total_gates': original_total_gates,
        'transpiled_total_gates': transpiled_total_gates,
        'gate_increase_percentage': gate_increase_percentage,
        'original_two_qubit_gates': original_two_qubit_count,
        'transpiled_two_qubit_gates': transpiled_two_qubit_count,
        'two_qubit_increase_percentage': two_qubit_increase,
        'swap_count': swap_count,
        'cx_count': transpiled_circuit.count_ops().get('cx', 0),
        'gate_counts': dict(transpiled_circuit.count_ops()),
        'execution_time': exec_time
    }

# Plot swap count overhead

In [ ]:
def plot_swap_count_comparison():
    """
    Plots SWAP count after transpilation vs number of qubits for different circuits and topologies.
    """
    plt.rcParams['figure.dpi'] = 300
    plt.rcParams['savefig.dpi'] = 600
    
    # Topologies to study
    topologies = ['linear', 'ring', 'grid', 'star']
    
    # Qubit count ranges
    regular_qubits = list(range(100, 1001, 100))  # 100 to 1000 with step 100 for all except TTN
    ttn_qubits = [2**i for i in range(3, 11)]     # 8, 16, 32, ..., 1024 for TTN
    
    # Define circuit types and their generators
    circuit_types = {
        'Kernel Linear': lambda n: generate_zz_linear_kernel(n).decompose(),
        'Kernel Circular': lambda n: generate_zz_circular_kernel(n).decompose(),
        'Kernel SCA': lambda n: generate_zz_sca_kernel(n).decompose(),
        'Kernel Pairwise': lambda n: generate_zz_pairwise_kernel(n).decompose(),
        'QNN Linear': lambda n: generate_qnn_linear(n).decompose(),
        'QNN Circular': lambda n: generate_qnn_circular(n).decompose(),
        'QNN SCA': lambda n: generate_qnn_sca(n).decompose(),
        'QNN Pairwise': lambda n: generate_qnn_pairwise(n).decompose(),
        'TTN': lambda n: create_tree_tensor_network(n),
        'GHZ': lambda n: generate_ghz_state(n)
    }
    
    markers = {name: ('None' if name == 'GHZ' else style) for name, style in 
              zip(circuit_types.keys(), ['o', 's', 'D', 'v', '^', '<', '>', 'p', '*', 'X'])}
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(circuit_types)))
    
    line_styles = {name: (':' if name == 'GHZ' else '-') for name in circuit_types.keys()}
    
    line_widths = {name: (2.5 if name == 'GHZ' else 1.5) for name in circuit_types.keys()}
    
    fig, axes = plt.subplots(1, 4, figsize=(12, 2.7))
    
    circuit_data = {}
    
    for i, (circuit_name, circuit_generator) in enumerate(circuit_types.items()):
        circuit_data[circuit_name] = {}
        
        # Select qubit range for the circuit
        if circuit_name == 'TTN':
            qubit_range = ttn_qubits
        else:
            qubit_range = regular_qubits
        
        for topology in topologies:
            swap_counts = []
            
            for num_qubits in qubit_range:
                try:
                    circuit = circuit_generator(num_qubits)
                    coupling_map = create_topology(topology, num_qubits)
                    metrics = analyze_transpiled_circuit(circuit, coupling_map)
                    swap_count = metrics['swap_count']
                    swap_counts.append(swap_count)
                    
                except Exception as e:
                    print(f"Error processing {circuit_name} with {num_qubits} qubits on {topology}: {str(e)}")
                    swap_counts.append(np.nan)
            
            # Results
            circuit_data[circuit_name][topology] = {'qubits': qubit_range, 'swap_counts': swap_counts}
    
    # Plot the results for each topology
    for t, topology in enumerate(topologies):
        ax = axes[t]
        circuit_names = list(circuit_data.keys())
        
        for circuit_name in circuit_names:
            data = circuit_data[circuit_name]
            i = list(circuit_types.keys()).index(circuit_name)
            if topology in data.keys():
                # GHZ is just a dotted line as a non-QML circuit
                if circuit_name == 'GHZ':
                    ax.plot(
                        data[topology]['qubits'], 
                        data[topology]['swap_counts'],
                        marker='None',
                        color=colors[i],
                        label=circuit_name,
                        linestyle=':',  
                        linewidth=2.5,  
                    )
                else:
                    # Normal plotting for all other circuits
                    ax.plot(
                        data[topology]['qubits'], 
                        data[topology]['swap_counts'],
                        marker=markers[circuit_name],
                        color=colors[i],
                        label=circuit_name,
                        linestyle=line_styles[circuit_name],
                        linewidth=line_widths[circuit_name],
                        markersize=5
                    )
        
        ax.set_xlabel('Number of Qubits', fontweight='bold', fontsize=10)
        if t == 0: 
            ax.set_ylabel('Number of SWAPs', fontweight='bold', fontsize=10)
        
        ax.set_title(f'{topology.capitalize()} Topology', fontweight='bold', fontsize=12)
        ax.tick_params(axis='both', which='major', labelsize=8)
        ax.grid(True, alpha=0.3, linestyle='--')

    handles, labels = [], []
    for ax in axes:
        h, l = ax.get_legend_handles_labels()
        for handle, label in zip(h, l):
            if label not in labels:
                handles.append(handle)
                labels.append(label)

    leg = fig.legend(handles, labels, 
               loc='lower center', 
               bbox_to_anchor=(0.5, 0.01),
               ncol=10,
               fontsize=8, 
               frameon=True,
               framealpha=0.8,
               columnspacing=0.7, 
               handletextpad=0.4)

    plt.tight_layout()
    plt.subplots_adjust(bottom=0.28, wspace=0.25)  # Using your optimal value
    
    # Save figures in different formats
    # plt.savefig('swap_count_comparison.png', dpi=600, bbox_inches='tight', format='png')
    plt.savefig('swap_count_comparison.pdf', dpi=600, bbox_inches='tight', format='pdf')
    # plt.savefig('swap_count_comparison.svg', dpi=600, bbox_inches='tight', format='svg')
    plt.show()
    
    return fig

# Final Implementation
swap_count_fig = plot_swap_count_comparison()

# Plot all circuit metrics

In [ ]:
def plot_metrics_by_topology():
    """
    Plots various metrics after transpilation vs number of qubits for different circuits and topologies.
    """
    plt.rcParams['figure.dpi'] = 300
    plt.rcParams['savefig.dpi'] = 600
    
    # Topologies to study
    topologies = ['linear', 'ring', 'grid', 'star']
    
    # Qubit count ranges
    regular_qubits = list(range(100, 1001, 100))  # 100 to 1000 with step 100 for all except TTN
    ttn_qubits = [2**i for i in range(3, 11)]     # 8, 16, 32, ..., 1024 for TTN
    
    circuit_types = {
        'Kernel Linear': lambda n: generate_zz_linear_kernel(n).decompose(),
        'Kernel Circular': lambda n: generate_zz_circular_kernel(n).decompose(),
        'Kernel SCA': lambda n: generate_zz_sca_kernel(n).decompose(),
        'Kernel Pairwise': lambda n: generate_zz_pairwise_kernel(n).decompose(),
        'QNN Linear': lambda n: generate_qnn_linear(n).decompose(),
        'QNN Circular': lambda n: generate_qnn_circular(n).decompose(),
        'QNN SCA': lambda n: generate_qnn_sca(n).decompose(),
        'QNN Pairwise': lambda n: generate_qnn_pairwise(n).decompose(),
        'TTN': lambda n: create_tree_tensor_network(n),
        'GHZ': lambda n: generate_ghz_state(n)  # GHZ moved to the end
    }
    
    # Define the metrics to analyze
    metrics = [
        'transpiled_depth',
        'depth_increase_percentage',
        'transpiled_two_qubit_gates',
        'two_qubit_increase_percentage'
    ]
    
    # Log scale for percentage increase metrics
    log_scale_metrics = ['depth_increase_percentage', 'two_qubit_increase_percentage']
    
    metric_titles = {
        'transpiled_depth': 'Transpiled Circuit Depth',
        'depth_increase_percentage': 'Depth Increase (%)',
        'transpiled_two_qubit_gates': 'Two-Qubit Gate Count',
        'two_qubit_increase_percentage': 'Two-Qubit Gate Increase (%)'
    }
    
    markers = {name: ('None' if name == 'GHZ' else style) for name, style in 
              zip(circuit_types.keys(), ['o', 's', 'D', 'v', '^', '<', '>', 'p', '*', 'X'])}
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(circuit_types)))

    line_styles = {name: (':' if name == 'GHZ' else '-') for name in circuit_types.keys()}

    line_widths = {name: (2.5 if name == 'GHZ' else 1.5) for name in circuit_types.keys()}

    circuit_data = {}
    
    # Basis gates are u3 and cx only (no swap as swaps are transpiled into 3 cx gates)
    basis_gates = ['u3', 'cx']
    
    for i, (circuit_name, circuit_generator) in enumerate(circuit_types.items()):
        circuit_data[circuit_name] = {}
        
        # Qubit range for this circuit
        if circuit_name == 'TTN':
            qubit_range = ttn_qubits
        else:
            qubit_range = regular_qubits

        for topology in topologies:
            circuit_data[circuit_name][topology] = {'qubits': qubit_range}

            for metric in metrics:
                circuit_data[circuit_name][topology][metric] = []

            for num_qubits in qubit_range:
                
                try:

                    circuit = circuit_generator(num_qubits)
                    

                    coupling_map = create_topology(topology, num_qubits)

                    metrics_data = analyze_transpiled_circuit(circuit, coupling_map, basis_gates=basis_gates)
                    

                    for metric in metrics:
                        value = metrics_data[metric]
                    
                except Exception as e:
                    print(f"Error processing {circuit_name} with {num_qubits} qubits on {topology}: {str(e)}")
                    for metric in metrics:
                        circuit_data[circuit_name][topology][metric].append(np.nan)
    
    # Create one combined figure
    fig, axes = plt.subplots(4, 4, figsize=(16, 12))
    
    for t, topology in enumerate(topologies):
        for m, metric in enumerate(metrics):
            ax = axes[t, m]

            circuit_names = list(circuit_data.keys())
            
            for circuit_name in circuit_names:
                data = circuit_data[circuit_name]
                i = list(circuit_types.keys()).index(circuit_name)

                if topology in data.keys():
    
                    if circuit_name == 'GHZ':
                        ax.plot(
                            data[topology]['qubits'], 
                            data[topology][metric],
                            marker='None',
                            color=colors[i],
                            label=circuit_name,
                            linestyle=':',
                            linewidth=2.5,
                        )
                    else:
                        ax.plot(
                            data[topology]['qubits'], 
                            data[topology][metric],
                            marker=markers[circuit_name],
                            color=colors[i],
                            label=circuit_name,
                            linestyle=line_styles[circuit_name],
                            linewidth=line_widths[circuit_name],
                            markersize=4
                        )
            
            # Logarithmic scale for percentage metrics
            if metric in log_scale_metrics:
                ax.set_yscale('log')

            if t == len(topologies)-1:
                ax.set_xlabel('Number of Qubits', fontweight='bold', fontsize=10)
            else:
                ax.set_xlabel('')
                
            if m == 0:
                ax.set_ylabel(f'{topology.capitalize()} Topology', fontweight='bold', fontsize=10)

            if t == 0:
                ax.set_title(metric_titles[metric], fontweight='bold', fontsize=12)

            ax.tick_params(axis='both', which='major', labelsize=8)

            ax.grid(True, alpha=0.3, linestyle='--')

    plt.tight_layout()

    handles, labels = [], []
    for row in axes:
        for ax in row:
            h, l = ax.get_legend_handles_labels()
            for handle, label in zip(h, l):
                if label not in labels:
                    handles.append(handle)
                    labels.append(label)

    leg = fig.legend(handles, labels, 
           loc='lower center', 
           bbox_to_anchor=(0.5, 0.01),
           ncol=5,
           fontsize=10, 
           frameon=True,
           framealpha=0.8,
           columnspacing=0.7, 
           handletextpad=0.4)

    plt.subplots_adjust(bottom=0.10, hspace=0.3, wspace=0.25)
    
    # Save the figure in multiple formats
    # plt.savefig('all_metrics_comparison.png', dpi=600, bbox_inches='tight', bbox_extra_artists=[leg], format='png')
    plt.savefig('all_metrics_comparison.pdf', dpi=600, bbox_inches='tight', bbox_extra_artists=[leg], format='pdf')
    # plt.savefig('all_metrics_comparison.svg', dpi=600, bbox_inches='tight', bbox_extra_artists=[leg], format='svg')
    
    plt.show()
    
    return fig

# Function to generate the plot
all_metrics_fig = plot_metrics_by_topology()